In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet50
import time
import wandb
#
from leaf._core import LeafConfig

Initializing _core module...
_core module initialization complete!


In [2]:
config = LeafConfig()

In [3]:
config.add_server(
    server_name="gpu-server-1",
    username="root",
    hostname="77.104.167.149",
    port=40236,  # Updated to correct SSH port
)

Starting connection verification...
Attempting SSH connection...


OpenSSH_9.0p1, LibreSSL 3.3.6
debug1: Reading configuration data /Users/ryanmarr/.ssh/config
debug1: Reading configuration data /etc/ssh/ssh_config
debug1: /etc/ssh/ssh_config line 21: include /etc/ssh/ssh_config.d/* matched no files
debug1: /etc/ssh/ssh_config line 54: Applying options for *
debug1: Authenticator provider $SSH_SK_PROVIDER did not resolve; disabling
debug1: Connecting to 77.104.167.149 [77.104.167.149] port 40236.
debug1: fd 3 clearing O_NONBLOCK
debug1: Connection established.
debug1: identity file /Users/ryanmarr/.ssh/id_rsa type 0
debug1: identity file /Users/ryanmarr/.ssh/id_rsa-cert type -1
debug1: identity file /Users/ryanmarr/.ssh/id_ecdsa type -1
debug1: identity file /Users/ryanmarr/.ssh/id_ecdsa-cert type -1
debug1: identity file /Users/ryanmarr/.ssh/id_ecdsa_sk type -1
debug1: identity file /Users/ryanmarr/.ssh/id_ecdsa_sk-cert type -1
debug1: identity file /Users/ryanmarr/.ssh/id_ed25519 type -1
debug1: identity file /Users/ryanmarr/.ssh/id_ed25519-cert typ


SSH connection successful
SSH verification successful, proceeding to gRPC verification
Starting gRPC connection verification...
Connecting to 77.104.167.149:40236 as root
Testing SSH connection...
SSH connection test successful
SSH connection test successful
Checking for Docker installation...
Docker is already installed
Checking Docker daemon status...
Docker daemon is already running
Copying Docker files to remote server...
Docker files copied successfully
Starting Docker container...
Using Docker command: /usr/bin/docker
Docker version 28.1.1, build 4eba377
Cleaning up any existing container...
leaf-grpc-server
Current directory: /tmp/leaf-build
Listing files in current directory:
total 24
drwxrwxrwx  2 root root 4096 Jun 20 02:09 .
drwxrwxrwt 13 root root 4096 Jun 20 02:09 ..
-rw-r--r--  1 root root  955 Jun 20 02:09 Dockerfile
-rwxr-xr-x  1 root root 2100 Jun 20 02:09 docker-run.sh
-rw-r--r--  1 root root 1010 Jun 20 02:09 server_test.cpp
-rw-r--r--  1 root root  258 Jun 20 02:09

In [4]:
config.print_all_resources()


=== Available Servers and Resources ===

Server: gpu-server-1
Status: Connected
Type: Remote server
Username: root
Hostname: 77.104.167.149
Port: 40236
Cleaning up SSH tunnel (PID: 46724)...

Available Resources (1):
--------------------------------------------------

Resource 1:
  Name: NVIDIA GeForce GTX 1660
  Type: GPU
  Properties:
    free_memory: 5745 MiB

    total_memory: 6144 MiB

--------------------------------------------------

Server: localhost
Status: Connected
Type: Local machine

Available Resources (1):
--------------------------------------------------

Resource 1:
  Name: Apple M1
  Type: CPU
  Properties:

--------------------------------------------------

=== End of Server List ===



ities: ssh_fetch_identitylist: agent contains no identities
debug1: Will attempt key: /Users/ryanmarr/.ssh/id_rsa RSA SHA256:D3umZJK4FT5tyWP0RKrYpBTpLWKFzEGw5U5mPN4dsng
debug1: Will attempt key: /Users/ryanmarr/.ssh/id_ecdsa 
debug1: Will attempt key: /Users/ryanmarr/.ssh/id_ecdsa_sk 
debug1: Will attempt key: /Users/ryanmarr/.ssh/id_ed25519 
debug1: Will attempt key: /Users/ryanmarr/.ssh/id_ed25519_sk 
debug1: Will attempt key: /Users/ryanmarr/.ssh/id_xmss 
debug1: Will attempt key: /Users/ryanmarr/.ssh/id_dsa 
debug1: SSH2_MSG_EXT_INFO received
debug1: kex_input_ext_info: server-sig-algs=<ssh-ed25519,sk-ssh-ed25519@openssh.com,ssh-rsa,rsa-sha2-256,rsa-sha2-512,ssh-dss,ecdsa-sha2-nistp256,ecdsa-sha2-nistp384,ecdsa-sha2-nistp521,sk-ecdsa-sha2-nistp256@openssh.com,webauthn-sk-ecdsa-sha2-nistp256@openssh.com>
debug1: kex_input_ext_info: publickey-hostbound@openssh.com=<0>
debug1: SSH2_MSG_SERVICE_ACCEPT received
debug1: Authentications that can continue: publickey
debug1: Next authentica

sh: line 0: kill: (46724) - No such process


In [5]:
asdf

NameError: name 'asdf' is not defined

In [5]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 50
batch_size = 128
learning_rate = 0.001

# Data preprocessing
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [6]:
# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size,
                        shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                     download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=batch_size,
                       shuffle=False, num_workers=2)

In [ ]:
# Load pretrained ResNet-50 and modify for CIFAR-10
model = resnet50(pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()  # Remove maxpool as CIFAR-10 images are small
model.fc = nn.Linear(model.fc.in_features, 10)  # Change output to 10 classes
model = model.to(device)

In [8]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

In [ ]:
run = wandb.init(
    project="big-model-example"
)

In [10]:
# Training loop
def train():
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if (batch_idx + 1) % 100 == 0:
            print(f'Batch: {batch_idx + 1} | Loss: {running_loss/(batch_idx + 1):.3f} | '
                  f'Acc: {100.*correct/total:.2f}%')
    
    return running_loss/len(trainloader), 100.*correct/total

In [11]:
# Testing loop
def test():
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    return test_loss/len(testloader), 100.*correct/total

In [ ]:
# Main training loop
print('Starting training...')
best_acc = 0
for epoch in range(num_epochs):
    start_time = time.time()
    
    train_loss, train_acc = train()
    test_loss, test_acc = test()
    
    scheduler.step(test_loss)
    
    print(f'\nEpoch: {epoch + 1}/{num_epochs}')
    print(f'Time: {time.time() - start_time:.2f}s')
    print(f'Train Loss: {train_loss:.3f} | Train Acc: {train_acc:.2f}%')
    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc:.2f}%')
    
    # Save best model
    if test_acc > best_acc:
        print('Saving best model...')
        state = {
            'model': model.state_dict(),
            'acc': test_acc,
            'epoch': epoch,
        }
        torch.save(state, 'best_model.pth')
        best_acc = test_acc
print('Training completed!')